converte Folhas de pagamento recebidas em formato html para planilhas excell
registrando cada evento como uma linha

# Configuração do ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
import pandas as pd
import html
import os
import re
from tqdm.notebook import tqdm_notebook

In [70]:
try:
  from unidecode import unidecode
except:
  !pip install unidecode
  from unidecode import unidecode

# Pré Processamento

In [71]:
def clean_html(path):
  with open(path, 'r', encoding='latin-1') as file:
    arq_html = file.read()
  arq_html = arq_html.split('\n')
  # Salva apenas as linhas que tem 'class'
  # e aplica a formatação para adicionar acentos
  arq_html = [html.unescape(row) for row in arq_html if 'class' in row or 'CLASS' in row]
  # Remove as divs
  for i, row in enumerate(arq_html):
    pattern = r'<[^>]*>'
    new_row = re.sub(pattern, '', row)
    # Em algum
    new_row = new_row.replace('\xa0', ' ')
    arq_html[i] = new_row

  return arq_html

In [153]:
# Padroniza os campos textuais da folha, removendo
# acentos, cedilhas e espaços no começo e fim
# E formata os CNPJs
def padroniza(folha):

  def padroniza_palavra(word):
    word = unidecode(str(word))
    word = word.replace('ç', 'c')
    word = word.replace('Ç', 'C')
    word = word.strip()
    return word

  colunas = ['Empregador', 'Nome_Empregado', 'Cargo', 'Lotação']

  for coluna in colunas:
    folha[coluna] = folha[coluna].apply(padroniza_palavra)

  def ajusta_cnpj(cnpj):
    cnpj = str(cnpj)
    caracteres = [',', '.', '-', '/']
    for char in caracteres:
      cnpj = cnpj.replace(char, '')
    return f'{int(cnpj):014}'

  folha['CNPJ'] = folha['CNPJ'].apply(ajusta_cnpj)

  def ajusta_lotacao(lotacao):
    if lotacao[0].isnumeric():
      lotacao = lotacao[lotacao.find(' - ')+3:]
    return lotacao

  folha['Lotação'] = folha['Lotação'].apply(ajusta_lotacao)

  return folha

In [73]:
# Limpa os arquivos no diretório
def clear(html=True, xlsx=True):
  files = os.listdir()

  if html:
    for file in files:
      if '.html' in file or '.HTML' in file:
        os.remove(file)

  if xlsx:
    for file in files:
      if '.xlsx' in file or '.xls' in file:
        os.remove(file)

## CERUS, GA, GESTART, HISEG, PATRIMONIAIS, PRIMEE, UNICA, VIGON e SINGULAR

In [74]:
# Esté método funciona para folhas de pagamento das empresas dos grupos:
# CERUS, FIBRA, GA, GESTART, HISEG, PATRIMONIAIS, PRIMEE, UNICA e VIGON
def get_registros_from_html(path, nome_arq='FolhaPagamento'):

  # Abre o arquivo e remove informações desnecessárias do arquivo
  arq_html = clean_html(path)

  # Declaração de variáveis
  columns = ['Competência','Empregador', 'CNPJ', 'Código_Empregado','Nome_Empregado','Cargo', 'Admissão','Lotação','Código_Evento','Desc_Evento','Provento','Desconto']
  registros = pd.DataFrame(columns=columns)
  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  admissao = str()
  lotacao = str()
  cargo = str()
  cnpj = str()

  # Encontra o CNPJ e Empregador que são únicos para o arquivo
  for i, row in enumerate(arq_html):
    if row ==  'Empregador':
      # O empregador está uma linha após a palavra
      empregador = arq_html[i+1]
      cnpj = str(arq_html[i-1])
      # Remove alguns itens do CNPJ para padronização
      remover = ['CNPJ: ', '.', '/', '-']
      for item in remover:
        cnpj = cnpj.replace(item, '')
      break

  # Total de empregados que serão coletados:
  total_func = 0
  for row in arq_html:
    if row == 'Data e Assinatura':
      total_func += 1

  # A variável start define o começo dos dados de cada empregado
  start = 0

  for _ in tqdm_notebook(range(total_func), leave=False):

    # A coleta destas informações não é feita por uso de um 'for'
    # pois a depender da empresa, alguma informação pode mudar
    # de posição

    # Encontra a competência
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Competência':
        # A competência está uma linha após esta palavra
        competencia = arq_html[i+1]
        break
    # Encontra a lotação
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Lotação':
        # A lotação está uma linha após a palavra
        lotacao = arq_html[i+1]
        break
    # Encontra o cargo
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Cargo':
        # O cargo está uma linha após a palavra
        cargo = arq_html[i+1]
        break
    # Encontra a admissão
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Admissão':
        # A admissão está uma linha após a palavra
        admissao = arq_html[i+1]
        break
    # Encontra o código e nome do empregado
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Empregado':
        # O código e nome estão uma linha após a palavra
        cod_e_nome = arq_html[i+1].split()
        codigo_emp = cod_e_nome[0]
        nome_emp = ' '.join(cod_e_nome[1:])
        break
    # Guarda os eventos do funcionário
    index = 0
    eventos = list()
    registro = list()

    for i, row in enumerate(arq_html[start:], start=start):
      # Os proventos começam após a linha com 'Cod.'
      if row == 'Cod.':
        index = i+1
        break


    while arq_html[index] != 'Total de Proventos':
      registro.append(arq_html[index])
      index += 1

      if len(registro) == 5:
        # Verifica se o código é não nulo
        if arq_html[index-1]:
          eventos.append(registro)
          registro = list()
        # Caso seja nulo, os eventos acabaram
        else:
          break

    # Converte a lista de eventos para DataFrame
    eventos = pd.DataFrame(eventos)
    # Pode acontecer de aparecer um funcionário sem registros
    # caso esteja afastado, então será ignorado
    if not eventos.empty:
      # Larga a coluna com as referências
      eventos = eventos.drop(2, axis='columns')
      # Inverte a ordem as colunas
      eventos = eventos[eventos.columns[::-1]]
      # Atualiza o nome das colunas
      eventos.columns = ['Código', 'Descrição', 'Proventos', 'Descontos']


    for i in range(eventos.shape[0]):
      # Cria uma lista com todos os dados do registro
      registro = [competencia, empregador, cnpj, codigo_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Proventos'].iloc[i], eventos['Descontos'].iloc[i]])
      # Adiciona uma nova linha na tabela de registros
      index = len(registros)
      registros.loc[index] = registro

    # Encontra os dados do próximo funcionário
    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if row == 'Data e Assinatura':
        start = i
        break

  registros = padroniza(registros)

  # Salva a tabela como xlsx
  nome_arq = f'{nome_arq}.xlsx'
  #registros.to_excel(nome_arq, index=False)

  return registros

In [75]:
# Atualiza alguns códigos para evitar conflitos
# Nem sempre é necessário
def update_cods(df):
  # Relação de códigos antigos e novos
  novo = {
      '350': '99A',
      '388': '99B'
  }

  for i in range(df.shape[0]):
    # É convertido para string pois alguns vem como int
    codigo = str(df['Código_Evento'].iloc[i])
    if codigo in novo:
      df.loc[i, 'Código_Evento'] = novo[codigo]

  return df

## CONDONAL

In [76]:
# Este método serve para as folhas do grupo CONDONAL
def get_registros_from_html_condonal(path, nome_arq='FolhaPagamento'):

  # Abre o arquivo e remove informações desnecessárias do arquivo
  arq_html = clean_html(path)

  # Declaração de variáveis
  columns = ['Competência','Empregador', 'CNPJ', 'Código_Empregado','Nome_Empregado','Cargo', 'Admissão', 'Lotação','Código_Evento','Desc_Evento','Provento','Desconto']
  registros = pd.DataFrame(columns=columns)
  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  admissao = str()
  lotacao = str()
  cargo = str()
  cnpj = str()

  # Função usada para ajustar o formato das competências
  def get_date(row):
    correspondencia = {
      '01/2023':   'Janeiro de 2023',
      '02/2023': 'Fevereiro de 2023',
      '03/2023':     'Março de 2023',
      '04/2023':     'Abril de 2023',
      '05/2023':      'Maio de 2023',
      '06/2023':     'Junho de 2023',
      '07/2023':     'Julho de 2023',
      '08/2023':    'Agosto de 2023',
      '09/2023':  'Setembro de 2023',
      '10/2023':   'Outubro de 2023',
      '11/2023':  'Novembro de 2023',
      '12/2023':  'Dezembro de 2023'
    }
    return correspondencia[row]



  # Total de empregados que serão coletados:
  total_func = 0
  for row in arq_html:
    if row == 'EMPRESA':
      total_func += 1

  # A variável start define o começo dos dados de cada empregado
  start = 0

  for _ in tqdm_notebook(range(total_func), leave=False):

    # Encontra o empregador e a lotação
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'EMPRESA':
        empregador = arq_html[i+6]
        cnpj = arq_html[i+7]
        lotacao = arq_html[i+8]
        break

    # Encontra a competência
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'SEQUÊNCIA':
        codigo_emp = arq_html[i+1]
        nome_emp = arq_html[i+2]
        admissao = arq_html[i+3]
        cargo = arq_html[i+4]
        competencia = arq_html[i+17]+arq_html[i+18]+arq_html[i+19]
        competencia = get_date(competencia)
        break


    # Guarda os eventos do funcionário
    index = 0
    eventos = list()
    registro = list()

    for i, row in enumerate(arq_html[start:], start=start):
      # Os proventos começam após a linha com 'BATIDAS'
      if row == 'BATIDAS':
        index = i+1
        break

    # Não há nenhuma palavra chave que indique o fim dos eventos
    # o fim dos eventos é seguido por uma série de linhas em branco
    # então é feita uma verificação das linhas em branco, se chegar
    # em 15, a coleta é encerrada
    count_nuls = 0
    # Pode acontecer de no meio dos eventos, ao fim de um evento a
    # página acabar, então a próxima linha será 'Continua...'
    # neste caso, os valores lidos devem ser ignorados até que se
    # encontre uma linha com o valor 'BATIDAS', que indica a
    # continuação dos eventos
    conteudo = True
    while count_nuls < 15:
      if arq_html[index] == 'Continua...':
        conteudo = False
        registro = list()

      if conteudo:
        registro.append(arq_html[index])
        if len(registro) == 14:
          eventos.append(registro)
          registro = list()
        # Contagem linhas em branco
        if arq_html[index]:
          count_nuls = 0
        else:
          count_nuls += 1
      else:
        if arq_html[index] == 'BATIDAS':
          conteudo = True
      index += 1

    # Converte a lista de eventos para DataFrame
    eventos = pd.DataFrame(eventos)
    # Pode acontecer de aparecer um funcionário sem registros
    # caso esteja afastado, então será ignorado
    if not eventos.empty:
      # Guarda apenas as colunas necessárias
      eventos = eventos[[3, 5, 9, 11]]
      # Atualiza o nome das colunas
      eventos.columns = ['Código', 'Descrição', 'Proventos', 'Descontos']

    for i in range(eventos.shape[0]):
      # Cria uma lista com todos os dados do registro
      registro = [competencia, empregador, cnpj, codigo_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Proventos'].iloc[i], eventos['Descontos'].iloc[i]])
      # Adiciona uma nova linha na tabela de registros
      index = len(registros)
      registros.loc[index] = registro

    # Encontra os dados do próximo funcionário
    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if row == 'EMPRESA':
        start = i
        break

  registros = padroniza(registros)

  # Salva a tabela como xlsx
  nome_arq = f'{nome_arq}.xlsx'
  #registros.to_excel(nome_arq, index=False)

  return registros

## FIBRA

In [77]:
# Este método lê os registros de folhas de pagamento de arquivos excel
# As empresas do grupo FIBRA entregam neste formato
def get_registros_from_xlsx_fibra(path, competencia, empregador, cnpj, start, end, indice):
  # A Admissão vem no formato AAAA-MM-DD 00:00:00, está funcão deixa na forma DD/MM/AAAA
  def ajusta_admissao(data):
    return f'{data[8:10]}/{data[5:7]}/{data[:4]}'

  columns = ['Competência',	'Empregador',	'CNPJ', 'Código_Empregado',	'Nome_Empregado',	'Cargo', 'Admissão',	'Lotação',	'Código_Evento',	'Desc_Evento',	'Provento',	'Desconto']
  registros = pd.DataFrame(columns=columns)

  columns = ['Numero', 'Nome', 'Jorn.', 'Cargo', 'Admissão', 'SF', 'IR']
  nrows = end - start + 2
  arq = pd.read_excel(path, skiprows=start, nrows=nrows).fillna('')
  arq.columns = columns

  admissao = str()
  nome = str()
  codigo = str()
  cargo = str()
  index = 1
  registro = list()

  lotacao = arq['Numero'].iloc[0]
  # Percorre todos os usuários, o fim dos dados de cada usuários
  # é seguido do código do próximo
  while arq['Numero'].iloc[index][1:].isnumeric():
    eventos = pd.DataFrame(columns=['Código', 'Descrição', 'Provento', 'Desconto'])

    admissao = str(arq['Admissão'].iloc[index])
    codigo = arq['Numero'].iloc[index]
    nome = arq['Nome'].iloc[index]
    cargo = arq['Cargo'].iloc[index]

    admissao = ajusta_admissao(admissao)

    # Encontra o começo dos eventos
    for i in range(index, nrows):
      if arq['Numero'].iloc[i] == 'Eventos':
        index = i+1
        break

    # Este while coleta todos os eventos
    # Os eventos são encerrados pela palavra 'Totais'
    while arq['Numero'].iloc[index] != 'Totais':
      registro = list()
      # Codigo evento
      registro.append(arq['Numero'].iloc[index])
      # Descrição
      registro.append(arq['Nome'].iloc[index])
      # Provento
      registro.append(arq['Admissão'].iloc[index])
      # Desconto
      registro.append(arq['SF'].iloc[index])

      eventos.loc[len(eventos)] = registro
      index += 1
    index += 1

    #Para cada linha de eventos, cria um registro completo
    for i in range(eventos.shape[0]):
      registro = [competencia, empregador, cnpj, codigo, nome, cargo, admissao, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Provento'].iloc[i], eventos['Desconto'].iloc[i]])

      registros.loc[len(registros)] = registro


  registros = padroniza(registros)

  nome_arq = f'folha_parte_{indice:04}.xlsx'
  #registros.to_excel(nome_arq, index=False)

  return registros

## SOLUCAO

In [78]:
# Resgata os registros de arquivos .xlsx de empresas do grupo SOLUCAO
# O formato como as planilhas são recebidos não é aberto com pd.read_excel()
# é necessário manualmente abrir a planilha e clicar em 'Formatar como tabela'
def get_registros_from_xlsx_solucao(path):

  arq = pd.read_excel(path).fillna('')

  eventos = pd.DataFrame()
  columns = ['Competência',	'Empregador', 'CNPJ',	'Código_Empregado',	'Nome_Empregado',	'Cargo', 'Admissão', 'Lotação',	'Código_Evento',	'Desc_Evento',	'Provento',	'Desconto']
  registros = pd.DataFrame(columns=columns)

  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  admissao = str()
  lotacao = str()
  cargo = str()
  cnpj = str()

  index = 0
  start = 0
  end = int()
  proximo = bool()
  count_empregados = 0

  col_lotacao = str()
  col_descricoes = str()
  col_proventos = str()
  col_descontos = str()
  col_competencia = str()
  col_nome = str()

  # Definição das colunas que serão guardadas

  # Guarda o empregador que é um para o arquivo todo
  empregador = arq['Coluna3'].iloc[0]
  for coluna in arq.columns[4:]:
    if arq[coluna].iloc[1]:
      cnpj = arq[coluna].iloc[1]
      cnpj = f'{int(cnpj):014}'
      break

  # Encontra a coluna em que está a lotação
  for coluna in arq:
    if arq[coluna].iloc[4] == 'Departamento':
      col_lotacao = coluna
      break

  # Encontra a coluna em que estão as descricoes
  for coluna in arq:
    if arq[coluna].iloc[8] == 'Descrição':
      col_descricoes = coluna
      break

  # Encontra a coluna em que estão os proventos
  for coluna in arq:
    if arq[coluna].iloc[8] == 'Vencimentos':
      col_proventos = coluna
      break

  # Encontra a coluna em que estão os descontos
  for coluna in arq:
    if arq[coluna].iloc[8] == 'Descontos':
      col_descontos = coluna
      break

  # Encontra a coluna em que estão as competências
  for i, coluna in enumerate(arq):
    if arq[coluna].iloc[1] == 'Folha Mensal':
      col_competencia = arq.columns[i+1]
      break

  # Encontra a coluna em que está o nome
  for coluna in arq:
    if arq[coluna].iloc[4] == 'Nome do Funcionário':
      col_nome = coluna
      break

  # Encontra a coluna em que está a admissão
  proximo = False
  for coluna in arq:
    if arq[coluna].iloc[6]:
      if proximo:
        col_admissao = coluna
        break
      elif arq[coluna].iloc[6] == 'Admissão:':
        proximo = True

  # Guarda apenas as colunas necessárias e as renomeia
  colunas = ['Coluna2', col_descricoes, col_proventos, col_descontos, col_nome, col_admissao, col_lotacao, col_competencia]
  arq = arq[colunas]
  arq.columns = ['Código', 'Descrição', 'Provento', 'Desconto', 'Nome_Empregado', 'Admissão', 'Lotação', 'Competência']

  # Conta o total de empregados na folha

  for i in arq['Código'].tolist():
    if i == 'Código':
      count_empregados += 1

  # Início da coleta de dados

  for _ in tqdm_notebook(range(count_empregados), leave=False):

    # Encontra a palavra chave 'Código' que será chave para achar os outros dados
    start = index
    for i, row in enumerate(arq['Código'].tolist()[start:], start=start):
      if row == 'Código':
        index = i+1
        break
    # Coleta os outros dados que estão 'ao redor' da palavra 'Código'
    codigo_emp = arq['Código'].iloc[index-4]
    nome_emp = arq['Nome_Empregado'].iloc[index-4]
    cargo = arq['Nome_Empregado'].iloc[index-3]
    admissao = str(arq['Admissão'].iloc[index-3])
    lotacao = arq['Lotação'].iloc[index-4]
    competencia = arq['Competência'].iloc[index-7]

    admissao = f'{admissao[8:10]}/{admissao[5:7]}/{admissao[:4]}'
    # Coleta os eventos

    end = index
    # Procura o fim dos eventos
    while arq['Código'].iloc[end]:
      end += 1
    else:
      eventos = arq[['Código', 'Descrição', 'Provento', 'Desconto']].iloc[index:end]


    #Para cada linha de eventos, cria um registro completo
    for i in range(eventos.shape[0]):
      registro = [competencia, empregador, cnpj, codigo_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Provento'].iloc[i], eventos['Desconto'].iloc[i]])

      registros.loc[len(registros)] = registro

  registros = padroniza(registros)

  return registros

In [122]:
def ajusta_lotacaoes(folha):

  # Essa função foi criada pois na tabela de relações, o nome da empresa
  # em questão está incompleto, então é criada uma coluna nova na folha
  # com o nome do emregador, mas no caso de ser a empresa com o nome cortado,
  # será adicionado o nome cortado
  def ajusta_empregador(emp):
    if emp == 'MAIS SOLUCAO EM LOCACAO DE MAO DE OBRA INTEGRADA S.A.':
      return ' MAIS SOLUCAO EM LOCACAO DE MAO DE OBRA I'
    else:
      return emp


  folha = folha.drop_duplicates().fillna('').reset_index()
  # Cria uma coluna auxiliar com o nome dos empregadores pois o nome
  # de alguns na tabela de relaççoes é diferente
  folha['Empregador_temp'] = folha['Empregador']
  folha['Empregador_temp'] = folha['Empregador_temp'].apply(ajusta_empregador)

  relacoes = pd.read_excel('Tabela de departamento.xlsx').fillna('')

  #folha = ajusta_codigos_eventos(folha, relacoes)

  for i in tqdm_notebook(range(folha.shape[0])):
    empresa = folha['Empregador_temp'].iloc[i]
    codigo_lot = folha['Lotação'].iloc[i]

    nome_codigo = relacoes['Nome'][(relacoes['Empresa']==empresa) & (relacoes['Código']==int(codigo_lot))].tolist()[0]

    folha.at[i, 'Lotação'] = nome_codigo

  # Larga a coluna Empregador_temp
  folha = folha.drop('Empregador_temp', axis='columns')

  return folha

# Leitura dos arquivos

## CERUS, GA, GESTART, HISEG, PATRIMONIAIS, PRIMEE, SINGULAR, UNICA E VIGON

In [ ]:
# Lista todos os arquivos html no diretório
files = [file for file in os.listdir() if '.html' in file]
folha_geral = pd.DataFrame()
# Lê cada arquivo no diretório e junta todos em um único DataFrame
for file in tqdm_notebook(files):
  folha_temp = get_registros_from_html(file)
  folha_geral = pd.concat([folha_geral, folha_temp])
# Converte o DataFrame para excel
folha_geral.to_excel('Folha_PRIMEE_FACILITIES.xlsx', index=False)

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/488 [00:00<?, ?it/s]

  0%|          | 0/573 [00:00<?, ?it/s]

  0%|          | 0/531 [00:00<?, ?it/s]

  0%|          | 0/423 [00:00<?, ?it/s]

...

  0%|          | 0/434 [00:00<?, ?it/s]

  0%|          | 0/576 [00:00<?, ?it/s]

  0%|          | 0/455 [00:00<?, ?it/s]

  0%|          | 0/471 [00:00<?, ?it/s]

  0%|          | 0/568 [00:00<?, ?it/s]

## CONDONAL

In [ ]:
files = [file for file in os.listdir() if '.HTML' in file]
folha_geral = pd.DataFrame()

for file in tqdm_notebook(files):
  folha_temp = get_registros_from_html_condonal(file)
  folha_geral = pd.concat([folha_geral, folha_temp])

folha_geral.to_excel('Folha_CONDONAL.xlsx', index=False)

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/1662 [00:00<?, ?it/s]

  0%|          | 0/2020 [00:00<?, ?it/s]

  0%|          | 0/2558 [00:00<?, ?it/s]

...

  0%|          | 0/2476 [00:00<?, ?it/s]

  0%|          | 0/2320 [00:00<?, ?it/s]

  0%|          | 0/2356 [00:00<?, ?it/s]

  0%|          | 0/2152 [00:00<?, ?it/s]

  0%|          | 0/2434 [00:00<?, ?it/s]

## FIBRA

In [ ]:
empregador = 'FIBRA TECNOLOGIA'
cnpj = '43635413000155'
meses = {
    '012023.xls': 'Janeiro de 2023',
    '022023.xls': 'Fevereiro de 2023',
    '032023.xls': 'Março de 2023',
    '042023.xls': 'Abril de 2023',
    '052023.xls': 'Maio de 2023',
    '062023.xls': 'Junho de 2023',
    '072023.xls': 'Julho de 2023',
    '082023.xls': 'Agosto de 2023',
    '092023.xls': 'Setembro de 2023',
    '102023.xls': 'Outubro de 2023',
    '112023.xls': 'Novembro de 2023'
}

In [ ]:
folha_completa = pd.DataFrame()
for mes in tqdm_notebook(meses):
  folha_fibra = pd.read_excel(mes).fillna('')
  competencia = meses[mes]
  start = int()
  indice = 0
  end = 0
  for i in tqdm_notebook(range(2, folha_fibra.shape[0]), leave=False):
    if folha_fibra['Numero'].iloc[i] == 'Totais':
      if not folha_fibra['Numero'].iloc[i+1][1:].isnumeric():
        start = end + 1
        end = i
        folha_parte = get_registros_from_xlsx_fibra(mes, competencia, empregador, cnpj, start, end, indice)
        indice += 1
        folha_completa = pd.concat([folha_completa, folha_parte])

folha_completa.to_excel(f'Folha_{empregador}.xlsx', index=False)

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/2312 [00:00<?, ?it/s]

  0%|          | 0/2328 [00:00<?, ?it/s]

  0%|          | 0/2283 [00:00<?, ?it/s]

  0%|          | 0/2324 [00:00<?, ?it/s]

...

  0%|          | 0/2253 [00:00<?, ?it/s]

  0%|          | 0/2264 [00:00<?, ?it/s]

  0%|          | 0/2277 [00:00<?, ?it/s]

  0%|          | 0/2476 [00:00<?, ?it/s]

  0%|          | 0/2595 [00:00<?, ?it/s]

In [ ]:
clear()

## SOLUCAO

In [129]:
folha = pd.DataFrame()

files = [file for file in os.listdir() if 'Recibo' in file]

for file in tqdm_notebook(files):
  folha_temp = get_registros_from_xlsx_solucao(file)
  folha = pd.concat([folha, folha_temp])

folha = ajusta_lotacaoes(folha)
folha.to_excel('Folha_SUPORTE_DE_LOCACAO.xlsx', index=False)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/496 [00:00<?, ?it/s]

  0%|          | 0/746 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/1072 [00:00<?, ?it/s]

...

  0%|          | 0/278 [00:00<?, ?it/s]

  0%|          | 0/840 [00:00<?, ?it/s]

  0%|          | 0/1120 [00:00<?, ?it/s]

  0%|          | 0/328 [00:00<?, ?it/s]

  0%|          | 0/31071 [00:00<?, ?it/s]

# Junção de Arquivos

In [133]:
folha = pd.DataFrame()

files = [file for file in os.listdir() if '.xlsx' in file]

for file in tqdm_notebook(files):
  folha_temp = pd.read_excel(file)
  folha = pd.concat([folha, folha_temp])

folha = padroniza(folha)
folha.to_excel('Folha_GERAL.xlsx', index=False)

  0%|          | 0/12 [00:00<?, ?it/s]